# Background flux

In [ ]:
import numpy as np
from itertools import product

import scopesim as sim
from scopesim.source.source_templates import empty_sky

# Set local path
sim.rc.__config__["!SIM.file.local_packages_path"] = "../../"

args = [("open", "Ks", "open"),
        ("open", "H", "open"),
        ("J", "open", "open"),
        ("open", "open", "Br-gamma")]
ao_modes = ("SCAO", "MCAO")
im_modes = ("IMG_1.5mas", "IMG_4mas")

In [ ]:
combos = list(product(ao_modes, im_modes, args))

In [ ]:
def background(ao_mode, im_mode, filters):
    fw1, fw2, pw = filters
    cmds = sim.UserCommands(use_instrument="MICADO",
                            set_modes=[ao_mode, im_mode])
    micado = sim.OpticalTrain(cmds)

    micado["filter_wheel_1"].change_filter(fw1)
    micado["filter_wheel_2"].change_filter(fw2)
    micado["pupil_wheel"].change_filter(pw)

    # Pixel size is 0.14 of the 4mas version, hence fluxes are lower
    # bg_flux *= (1.5 / 4)**2
    
    src = empty_sky()
    micado.observe(src)
    implane = micado.image_planes[0].hdu.data       # e-/pixel/s

    return np.median(implane)

def filtername(filters):
    for fltr in filters:
        if fltr != "open":
            return fltr
    return fltr

In [ ]:
results = []
for combo in combos:
    a, m, fltrs = combo
    results.append((a, m, filtername(fltrs), background(*combo)))

In [ ]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

## Setup Table to show grouped results

In [ ]:
idx = pd.MultiIndex.from_tuples([r[:3] for r in results],
      names=["AO", "Mode", "Filter"]).reorder_levels(["AO", "Filter", "Mode"])
df = pd.DataFrame([r[-1] for r in results], columns=["e-/pixel/s"], index=idx)
df.sort_values(by=["AO", "Filter", "Mode"], ascending=[False, False, True])